In [1]:
import numpy as np

from RushHour4.setup import Map, Environment

In [2]:
map = Map(6, 9)
map.index
map.coordinate
map.view()

0,1,2,3,4,5,6,7,8
9,10,11,12,13,14,15,16,17
18,19,20,21,22,23,24,25,26
27,28,29,30,31,32,33,34,35
36,37,38,39,40,41,42,43,44
45,46,47,48,49,50,51,52,53


"( 0, 0 )","( 0, 1 )","( 0, 2 )","( 0, 3 )","( 0, 4 )","( 0, 5 )","( 0, 6 )","( 0, 7 )","( 0, 8 )"
"( 1, 0 )","( 1, 1 )","( 1, 2 )","( 1, 3 )","( 1, 4 )","( 1, 5 )","( 1, 6 )","( 1, 7 )","( 1, 8 )"
"( 2, 0 )","( 2, 1 )","( 2, 2 )","( 2, 3 )","( 2, 4 )","( 2, 5 )","( 2, 6 )","( 2, 7 )","( 2, 8 )"
"( 3, 0 )","( 3, 1 )","( 3, 2 )","( 3, 3 )","( 3, 4 )","( 3, 5 )","( 3, 6 )","( 3, 7 )","( 3, 8 )"
"( 4, 0 )","( 4, 1 )","( 4, 2 )","( 4, 3 )","( 4, 4 )","( 4, 5 )","( 4, 6 )","( 4, 7 )","( 4, 8 )"
"( 5, 0 )","( 5, 1 )","( 5, 2 )","( 5, 3 )","( 5, 4 )","( 5, 5 )","( 5, 6 )","( 5, 7 )","( 5, 8 )"


o,o,o,o,o,o,o,o,o
o,o,o,o,o,o,o,o,o
o,o,o,o,o,o,o,o,o
o,o,o,o,o,o,o,o,o
o,o,o,o,o,o,o,o,o
o,o,o,o,o,o,o,o,o


In [3]:
obstruct_ids = [12, 13, 21, 30, 20, 29, 33, 34, 42, 43, 7, 8, 23]
for position in obstruct_ids:
    map.obstruct(position, index=True)

map.view()
world = map.save()

o,o,o,o,o,o,o,x,x
o,o,o,x,x,o,o,o,o
o,o,x,x,o,x,o,o,o
o,o,x,x,o,o,x,x,o
o,o,o,o,o,o,x,x,o
o,o,o,o,o,o,o,o,o


In [4]:
# env = Environment(world)
env = Environment(map)

In [5]:
env.valid_actions(22, index=True)

['down']

In [6]:
env.down(22, index=True)

31

In [7]:
env.valid_actions(31, index=True)

['up', 'down', 'right']

In [8]:
for action in ['up', 'down', 'right']:
    print(env.next(action, 31, index=True))

22
40
32
